In [4]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

In [5]:
Peterson = pd.read_excel('ESI-_3_CF2_series/AFFF_PetersonBucket_3sigma-_series.xlsx')
onsite_1 = pd.read_excel('ESI-_3_CF2_series/onbase1_A_Peter_T04-_series.xlsx')
offsite_2 = pd.read_excel('ESI-_3_CF2_series/offbase2_Security-_series.xlsx')

Peterson.drop(columns=['stats', '#_pks', '#_assgns', '#_series'], inplace=True)
Peterson = Peterson[Peterson['category'].isin(['A', 'B', 'C'])]
onsite_1.drop(columns=['stats', '#_pks', '#_assgns', '#_series'], inplace=True)
onsite_1 = onsite_1[onsite_1['category'].isin(['A', 'B', 'C'])]
offsite_2.drop(columns=['stats', '#_pks', '#_assgns', '#_series'], inplace=True)
offsite_2 = offsite_2[offsite_2['category'].isin(['A', 'B', 'C'])]

Peterson['Source'] = 'AFFF'
onsite_1['Source'] = 'on-site 1'
offsite_2['Source'] = 'off-site 2'

Peterson = Peterson.drop_duplicates(subset='exper_mz')
onsite_1 = onsite_1.drop_duplicates(subset='exper_mz')
offsite_2 = offsite_2.drop_duplicates(subset='exper_mz')

master_df = pd.concat([Peterson, onsite_1, offsite_2], ignore_index=True)

master_df['Source'] = master_df.groupby('formula')['Source'].transform(lambda x: ' & '.join(x))

master_df.reset_index(drop=True, inplace=True)

master_df['O/C'] = master_df['O'] / master_df['C']
master_df['F/C'] = master_df['F'] / master_df['C']
master_df['abund_norm'] = master_df['abund'] / master_df['abund'].max()
master_df['NOSC'] = 4 - ((4*master_df['C'] - master_df['F'] + master_df['H'] - 2*master_df['O'] + 4*master_df['S'] 
                          - 3*master_df['N'] - master_df['Cl'] + 5*master_df['P']) / master_df['C'])
master_df['mass_defect'] = master_df['lib_mass'] - master_df['lib_mass'].round()
master_df['Abund. ($log_{10}$)'] = np.log10(master_df['abund_norm'])
master_df['Category'] = master_df['category']

master_df.loc[0, 'stats'] = 'AFFF'
master_df.loc[1, 'stats'] = 'on-1'
master_df.loc[2, 'stats'] = 'off-2'
master_df.loc[3, 'stats'] = 'AFFF & on-1'
master_df.loc[4, 'stats'] = 'AFFF & off-2'
master_df.loc[5, 'stats'] = 'on-1 & off-2'
master_df.loc[6, 'stats'] = 'AFFF & on-1 & off-2'

master_df_catABC = master_df.drop_duplicates(subset='formula')
master_df.loc[0, 'category ABC'] = master_df_catABC[master_df_catABC['Source'] == 'AFFF']['exper_mz'].nunique()
master_df.loc[1, 'category ABC'] = master_df_catABC[master_df_catABC['Source'] == 'on-site 1']['exper_mz'].nunique()
master_df.loc[2, 'category ABC'] = master_df_catABC[master_df_catABC['Source'] == 'off-site 2']['exper_mz'].nunique()
master_df.loc[3, 'category ABC'] = master_df_catABC[master_df_catABC['Source'] == 'AFFF & on-site 1']['exper_mz'].nunique()
master_df.loc[4, 'category ABC'] = master_df_catABC[master_df_catABC['Source'] == 'AFFF & off-site 2']['exper_mz'].nunique()
master_df.loc[5, 'category ABC'] = master_df_catABC[master_df_catABC['Source'] == 'on-site 1 & off-site 2']['exper_mz'].nunique()
master_df.loc[6, 'category ABC'] = master_df_catABC[master_df_catABC['Source'] == 'AFFF & on-site 1 & off-site 2']['exper_mz'].nunique()

master_df_catA = master_df_catABC[(master_df_catABC['Category'] == 'A')]
master_df.loc[0, 'category A'] = master_df_catA[master_df_catA['Source'] == 'AFFF']['exper_mz'].nunique()
master_df.loc[1, 'category A'] = master_df_catA[master_df_catA['Source'] == 'on-site 1']['exper_mz'].nunique()
master_df.loc[2, 'category A'] = master_df_catA[master_df_catA['Source'] == 'off-site 2']['exper_mz'].nunique()
master_df.loc[3, 'category A'] = master_df_catA[master_df_catA['Source'] == 'AFFF & on-site 1']['exper_mz'].nunique()
master_df.loc[4, 'category A'] = master_df_catA[master_df_catA['Source'] == 'AFFF & off-site 2']['exper_mz'].nunique()
master_df.loc[5, 'category A'] = master_df_catA[master_df_catA['Source'] == 'on-site 1 & off-site 2']['exper_mz'].nunique()
master_df.loc[6, 'category A'] = master_df_catA[master_df_catA['Source'] == 'AFFF & on-site 1 & off-site 2']['exper_mz'].nunique()

master_df.to_csv('ESI-_8_differential_VKDs/differential_VKD_worksheet.csv')
master_df_copy = master_df.drop_duplicates(subset='formula')
master_df_copy.to_excel('ESI-_8_differential_VKDs/differential_VKD_chemical_list.xlsx')

In [6]:
# 0. van Krevlen plot for O/C vs. F/C, for overlapping species:

master_df['Source'] = master_df['Source'].replace({'AFFF & on-site 1': 'AFFF & on-1', 
                                                   'AFFF & off-site 2': 'AFFF & off-2',
                                                   'on-site 1 & off-site 2': 'on-1 & off-2',
                                                   'AFFF & on-site 1 & off-site 2': 'all'})

## Color scheme
sns.set_theme(style='white')
palette_dict0 = {'AFFF & on-1':'#FF7043', 'AFFF & off-2':'#8E44AD', 'on-1 & off-2':'#669900', 'all':'#969696'}

hue_order_list0 = []
if 'AFFF & on-1' in master_df['Source'].values:
    hue_order_list0.append('AFFF & on-1')
if 'AFFF & off-2' in master_df['Source'].values:
    hue_order_list0.append('AFFF & off-2')
if 'on-1 & off-2' in master_df['Source'].values:
    hue_order_list0.append('on-1 & off-2')
if 'all' in master_df['Source'].values:
    hue_order_list0.append('all') 

marker_dict0 = {'known': 'o', 'unknown': 'X'}    
master_df['Category'] = master_df['Category'].replace({'A': 'known'})
master_df['Category'] = master_df['Category'].replace({'B': 'unknown','C': 'unknown'})

## Plotting  
g = sns.relplot(data=master_df, x=master_df['O/C'], y=master_df['F/C'], 
                size='Abund. ($log_{10}$)', hue='Source', hue_order=hue_order_list0, 
                palette=palette_dict0, markers=marker_dict0, style='Category', sizes=(3, 30), facet_kws=dict(despine=False), 
                height=5, aspect=1)

g.set(xscale='linear', yscale='linear')
g.ax.set_xlim([-0.1, 1.5])  
g.ax.set_ylim([-0.1, 3.0])
plt.xlabel('O/C', fontsize=15)
plt.ylabel('F/C', fontsize=15) 

x_start, x_end = g.ax.get_xlim()
y_start, y_end = g.ax.get_ylim()
g.ax.xaxis.set_ticks(np.arange(x_start, x_end, 0.01), minor=True)
g.ax.yaxis.set_ticks(np.arange(y_start, y_end, 0.01), minor=True)
g.ax.grid(True, which='major', linewidth=.2, color='silver')
g._legend.remove()

plt.legend(prop={'size': 7}, loc='lower right')   

plt.savefig('ESI-_8_differential_VKDs/differential_OCFC_overlap.png', dpi=600, bbox_inches='tight')
plt.close()

In [7]:
# 1. van Krevlen plot for O/C vs. F/C:

master_df['Source'] = master_df['Source'].replace({'AFFF & on-1': 'multiple', 
                                                   'AFFF & off-2': 'multiple',
                                                   'on-1 & off-2': 'multiple',
                                                   'AFFF & on-1 & off-2': 'multiple'})

## Color scheme
sns.set_theme(style='white')
palette_dict = {'AFFF':'#D60093', 'on-site 1':'#FFCE44', 'off-site 2':'#4472C4', 'multiple':'#969696'}

hue_order_list = []
if 'AFFF' in master_df['Source'].values:
    hue_order_list.append('AFFF')
if 'on-site 1' in master_df['Source'].values:
    hue_order_list.append('on-site 1')
if 'off-site 2' in master_df['Source'].values:
    hue_order_list.append('off-site 2')
if 'multiple' in master_df['Source'].values:
    hue_order_list.append('multiple') 

marker_dict = {'known': 'o', 'unknown': 'X'}    
master_df['Category'] = master_df['Category'].replace({'A': 'known'})
master_df['Category'] = master_df['Category'].replace({'B': 'unknown','C': 'unknown'})

## Plotting  
g = sns.relplot(data=master_df, x=master_df['O/C'], y=master_df['F/C'], 
                size='Abund. ($log_{10}$)', hue='Source', hue_order=hue_order_list, 
                palette=palette_dict, markers=marker_dict, style='Category', sizes=(3, 30), facet_kws=dict(despine=False), 
                height=5, aspect=1)

g.set(xscale='linear', yscale='linear')
g.ax.set_xlim([-0.1, 1.5])  
g.ax.set_ylim([-0.1, 3.0])
plt.xlabel('O/C', fontsize=15)
plt.ylabel('F/C', fontsize=15) 

x_start, x_end = g.ax.get_xlim()
y_start, y_end = g.ax.get_ylim()
g.ax.xaxis.set_ticks(np.arange(x_start, x_end, 0.01), minor=True)
g.ax.yaxis.set_ticks(np.arange(y_start, y_end, 0.01), minor=True)
g.ax.grid(True, which='major', linewidth=.2, color='silver')
g._legend.remove()

plt.legend(prop={'size': 7}, loc='lower right')   

plt.savefig('ESI-_8_differential_VKDs/differential_OCFC.png', dpi=600, bbox_inches='tight')
plt.close()

In [8]:
# 2. van Krevlen plot for Mass vs. NOSC:      
g = sns.relplot(data=master_df, x=master_df['exper_mz'], y=master_df['NOSC'], 
                size='Abund. ($log_{10}$)', hue='Source', hue_order=hue_order_list, 
                palette=palette_dict, markers=marker_dict, style='Category', sizes=(3, 30), facet_kws=dict(despine=False),
                height=5, aspect=1.02)
    
g.set(xscale='linear', yscale='linear')
g.ax.set_xlim([150, 1150])  
g.ax.set_ylim([-2.5, 4])
g.ax.set_xticks([200, 300, 400, 500, 600, 700, 800, 900, 1000, 1100])  
plt.xlabel('Mass (Da)', fontsize=15)
plt.ylabel('NOSC', fontsize=15) 
    
x_start, x_end = g.ax.get_xlim()
y_start, y_end = g.ax.get_ylim()
g.ax.xaxis.set_ticks(np.arange(x_start, x_end, 5), minor=True)
g.ax.yaxis.set_ticks(np.arange(y_start, y_end, 0.05), minor=True)
g.ax.grid(True, which='major', linewidth=.2, color='silver')
# g.ax.grid(True, which='minor', linewidth=.2)
# g.despine(left=True, bottom=True)
g._legend.remove()

plt.savefig('ESI-_8_differential_VKDs/differential_NOSC.png', dpi=600, bbox_inches='tight')
plt.close()